### **Import necesary tools**

In [1]:
from transformers import pipeline, set_seed
import torch

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

responses = generator("Inima imi spune", max_length=128, num_return_sequences=5)

for response in responses:
  print(response["generated_text"])
  print("-----")

Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Inima imi spune i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nihilo, nee aliquo me i nih
-----
Inima imi spune is the second of three stories from the same storybook and it is also the first to feature the new villain she was introduced in. She was introduced in the storybook for the first time in the main story, "The Dark Witch".

She has been brought back in the main story for the first time in the storybook for the first time in the main story, "The Dark Witch".

In the second story, "The Demon King is abou

In [3]:
base_model = "readerbench/RoGPT2-medium"  # or RoGPT2-base / RoGPT2-large

### **Reformat the dataset as poem-level text**

In [4]:
import re
from datasets import load_dataset, Dataset

raw = load_dataset("json", data_files={"train": "results.jsonl"})["train"]
df = raw.to_pandas()

# Sort to reconstruct poems correctly
df = df.sort_values(["title", "verse_index"])

def normalize_verse(v: str):
    v = v.replace("\r\n", "\n").strip()
    # treat '@' as stanza boundary marker (often appears as " @" at end)
    stanza_break = "@" in v
    v = v.replace("@", "").strip()
    return v, stanza_break

poems = []
for title, g in df.groupby("title", sort=False):
    lines = [f"<|title|> {title}"]  # title conditioning (recommended)
    for v in g["verse"].tolist():
        line, br = normalize_verse(v)
        if line:
            lines.append(line)
        if br:
            # insert stanza token instead of blank line
            lines.append("<|stanza|>")
    poem_text = "\n".join(lines).strip()
    poems.append(poem_text)

poem_ds = Dataset.from_dict({"text": poems})
print(poem_ds[0]["text"][:300])
print("Num poems:", len(poem_ds))

Generating train split: 2295 examples [00:00, 525777.13 examples/s]

<|title|> ***
Ce fel de tren marfar ești tu
dacă ți-e trupul meu șină de carne;
Ce fel de măr ești tu
dacă ți-e ramură viața mea?
<|stanza|>
Eu locuiesc într-un tril
de privighetoare
Dorm cu ceafa pe nota Do
și-mi încălț piciorul
într-un saxofon
<|stanza|>
Du-te, îmi strigă ciocanul,
du-te,
du-te id
Num poems: 139


### **Import Tokenizer**

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Set the padding token to match the end-of-sequence (EOS) token
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# Add a special token for title to make prompting stable
specials = {"additional_special_tokens": ["<|title|>", "<|stanza|>"]}
num_added = tokenizer.add_special_tokens(specials)

In [8]:
def preprocess(examples):
    # Add EOS to mark poem end (as in the article) :contentReference[oaicite:6]{index=6}
    return {"text": [t.strip() + tokenizer.eos_token for t in examples["text"]]}

def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=False, add_special_tokens=True)

In [9]:
poem_ds = poem_ds.map(preprocess, batched=True)
tok = poem_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

block_size = 256  # 512 also fine; 256 is easier on memory
def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_len = len(concatenated["input_ids"])
    total_len = (total_len // block_size) * block_size
    return {
        k: [t[i:i+block_size] for i in range(0, total_len, block_size)]
        for k, t in concatenated.items()
    }

lm_ds = tok.map(group_texts, batched=True, batch_size=1000)

Map: 100%|██████████| 139/139 [00:00<00:00, 9602.06 examples/s]


#### **Train with Trainer**

In [11]:
import torch
from transformers import (
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

model = AutoModelForCausalLM.from_pretrained(base_model)

# If we added tokens (e.g., <|title|>), resize embeddings
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))

model.config.bos_token_id = tokenizer.bos_token_id or tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.eos_token_id

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM (as in the article) :contentReference[oaicite:7]{index=7}
)

# Optional: split train/eval for perplexity tracking
split = lm_ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = split["train"], split["test"]

use_fp16 = torch.cuda.is_available()

training_args = TrainingArguments(
    output_dir="./nichita-Ro-gpt2",
    num_train_epochs=50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    save_strategy="epoch",
    logging_steps=50,
    eval_strategy="no",
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("./nichita-Ro-gpt2", safe_serialization=True)
tokenizer.save_pretrained("./nichita-Ro-gpt2")

Step,Training Loss
50,4.544300
100,3.244300
150,2.650900
200,2.212300
250,1.888500
300,1.612800
350,1.415700
400,1.261200
450,1.164400
500,1.085900


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be 

('./nichita-Ro-gpt2/tokenizer_config.json',
 './nichita-Ro-gpt2/special_tokens_map.json',
 './nichita-Ro-gpt2/vocab.json',
 './nichita-Ro-gpt2/merges.txt',
 './nichita-Ro-gpt2/added_tokens.json',
 './nichita-Ro-gpt2/tokenizer.json')

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_DIR = "./nichita-Ro-gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# IMPORTANT for GPT-2 style models
tokenizer.pad_token = tokenizer.eos_token

def generate(prompt,
             max_new_tokens=150,
             temperature=0.8, top_p=0.90, top_k=40,
             repetition_penalty=1.2, no_repeat_ngram_size=3,
             min_new_tokens=30, num_return_sequences=1, do_sample=True):
    import re as _re
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            min_new_tokens=min_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    texts = tokenizer.batch_decode(out, skip_special_tokens=False)
    formatted = []
    for t in texts:
        cont = t[len(prompt):] if t.lower().startswith(prompt.lower()) else t
        cont = cont.replace("<|stanza|>", "\n\n")
        cont = _re.sub(r'\r\n', '\n', cont)
        cont = _re.sub(r'\n\s*\n+', '\n\n', cont).strip()
        lines = [ln.rstrip() for ln in cont.splitlines() if ln.strip()]
        formatted.append("\n".join(lines))
    return formatted[0] if num_return_sequences == 1 else formatted


In [13]:
def print_poem(output):
    if isinstance(output, list):
        for i, p in enumerate(output, 1):
            if len(output) > 1:
                print(f"--- Poem {i} ---")
            print(p.strip())
            print()
    else:
        print(output.strip())

In [14]:
file_result_output = "medium_rogpt2_results.txt"

with open(file_result_output, "w", encoding="utf-8") as f_out:
    prompts = [
        "Pe lângă plopii fără soț",
        "Mă-ntorc mereu la tine",
        "A fost o vreme când",
        "Nu credeam că va veni",
        "Sub cerul plin de stele",
    ]
    for prompt in prompts:
        poem = generate(prompt, num_return_sequences=2)
        f_out.write(f"--- Prompt: {prompt} ---\n")
        if isinstance(poem, list):
            for i, p in enumerate(poem, 1):
                f_out.write(f"--- Poem {i} ---\n")
                f_out.write(p.rstrip() + "\n\n")
        else:
            f_out.write(poem.rstrip() + "\n\n")
        f_out.write("END OF POEM\n\n")

In [ ]:
result = generate("Dragoste\n")
print_poem(result)

o să fim ca și cum am fi
și dragostea ne va face la fel
și trupurile noastre o s-ajungă până în rai.
Sufletele noastre un singur lucru vor
să fie, să iubească.
Noi suntem un singur trup
pentru că sângele apă nu se face.
Apă nu există decât în pahare
și sângele e cea mai puternică substanță
a universului.
Umbrele sunt umbre ale lucrurilor
care sunt
esențe tari, scumpe
cu care ochiul nostru privește
pe cei din jurul lui, cu care își‐mparte el însuși existența.
Haideți să ne iubim
ca niște regi!
Ca niște prinți, ca niște princese
de sânge care curg deasupra capetelor


Poetul si soldatul
Din zori și până-n seară, cu toții suntem eroi. ___________ Cu fruntea sus și cu pieptul în vânt. ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ─────── ────

In [ ]:
print(generate("durerea iubirii apuse\n"))

și în ea o flacără galbenă, albastră
atât de dureroasă încât îmi smulg dinții din gură
și-o scuip în obrazul rece ca pământul.
De ce să trăiesc într-un pământ atât de străin?
Dar nu vreau să mor în zadar
de vreme ce mi s-a dat șansa aceasta
să fiu fericit.
Niciodată n-am crezut că există oameni ursuzi, dar
când am văzut un om frumos zburând cu aripi
cu mult deasupra pământului l-am privit cu uimire
dar când am coborât privirea spre călcâiul lui,
mi-au venit în ochi lacrimi de sânge.
...Și deodată am înțeles că eu sunt
cel urât, cel urât dintr-o bucată,
ceilalți


Inima ma doare
Iarăși am plecat din viață,֔de parcă aș fi făcut parte din nou din viață.─┤ O lacrimă se rostogolește──O altă lacrimă se rostogolte──Și-n fiecare lacrimă se oglindește aceeași speranță志<|endoftext|>

In [ ]:
print(generate("cer si stele\n"))

și-n mine, și-n lume.
...și mă rog de tine
să fii bună
ca sarea-ntr-un burete.
Și să ai milă de cel sărac
atât cât are el chef
și să nu știi niciodată că altul are mai mult
decât tine!
Prea te prefaci că uiți, și te-apuci de năzbâtii,
de foame și sete,
prea te prefacești c-ai fi viu,
plin de boli, sărac, bolnav...
Dar eu sunt fericit când știu că tu
ești într-adevăr omeneşte
și chiar dacă ți se văd coastele,
nu înseamnă că ești mort.
Abia aștept să mor și să fiu văzut
în


Ganduri de seara
Plouă, plouă cu stele.
Când m-oi trezi din somn,
ca să mă uit la tine,
clipită din pleoape,
ce cald mi se va părea că ești,
și cât de frumoasă eşti,
de parcă ai fi aievea.

Mi s-a părut că te văd,
cum plutești deasupra mea,
cu inima smulsă din piept și totuși
mi-ai spus: "Tu nu mai poți trăi
în mine".
Văzduhul e ca apa,
dar în el sunt eu, iar
eu sunt tu.
Nu există nici un fel de liniște
între noi, ci doar o respirație
din care curge sângele meu,
care, pentru mine, este viață.

Sunt viu, dar în mine ești tu
și viața mea e viață
pentru tine.
...
Eu trăiesc, dar tu ești tu
iar eu sunt tu.

Mă întind, tu îmi întinzi mâna